In [10]:
from core.multi_case_modeller import MultiCaseModeller
from sixgill.definitions import ModelComponents, Parameters
from sixgill.pipesim import Model, Units
from core import ExcelInputError
from project import (
    get_string_values_from_class,
    generate_dict_from_class,
    get_class_by_name,
)
import pandas as pd
from core.network_simulation import NetworkSimulator

In [11]:
m = "trail.pips"

In [ ]:
ns = NetworkSimulator(m)

In [ ]:
ns.run_existing_model()